# Generate Golden Test Data

**Purpose**: Print data at each stage of the pipeline so you can copy-paste into golden tests.

**Steps**:
1. Run each cell
2. Copy the output
3. Paste into `tests/test_golden_output.py`

**Simple!** output raw data，no fancy formatting。

## Setup

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Import our modules
from datasets import load_dataset
from src.data.dataset import convert_to_conversation_format
from src.data.collators import restore_images_in_conversations

print("✅ Imports ready")

✅ Imports ready


## Stage 1: Raw Sample from Dataset

Load the first training sample - this is what comes directly from HuggingFace.

In [ ]:
# Load dataset (first sample only)
ds = load_dataset("openfoodfacts/nutrition-table-detection", split="train[:1]", streaming=False)
raw_sample = ds[0]

# Output the raw sample
# You'll copy this to understand the raw format
raw_sample

{'image_id': '0009800892204_1',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2592x1944>,
 'width': 2592,
 'height': 1944,
 'meta': {'barcode': '0009800892204',
  'off_image_id': '1',
  'image_url': 'https://static.openfoodfacts.org/images/products/000/980/089/2204/1.jpg'},
 'objects': {'bbox': [[0.057098764926195145,
    0.014274691231548786,
    0.603501558303833,
    0.991126537322998]],
  'category_id': [0],
  'category_name': ['nutrition-table']}}

### 📋 What to copy from above:
- Note the keys: `image`, `objects` with `bbox` and `category_name`
- Note the bbox format: `[y_min, x_min, y_max, x_max]` normalized [0,1]
- Note it's a PIL Image

## Stage 2: After convert_to_conversation_format()

This is what the data looks like after preprocessing (with `IMAGE_PLACEHOLDER`).

In [ ]:
# Convert to conversation format
converted = convert_to_conversation_format(raw_sample)

# Output the converted sample
converted

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]},
  {'role': 'user',
   'content': [{'type': 'image', 'image': 'IMAGE_PLACEHOLDER'},
    {'type': 'text',
     'text': 'Detect the bounding box of the nutrition table.'}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(14,57),(991,603)<|box_end|>'}]}],
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2592x1944>}

### 📋 What to copy from above:
- Copy the entire `converted` dict
- Note: `'image'` is still PIL Image
- Note: `'messages'` has `'IMAGE_PLACEHOLDER'` string
- This is what gets saved to disk (serializable)

## Stage 3: Batch of Samples (2 DIFFERENT samples)

This is what a real batch looks like - each item is a DIFFERENT sample.

In [ ]:
# Load 2 different samples for real batch
ds_batch = load_dataset("openfoodfacts/nutrition-table-detection", split="train[:2]", streaming=False)

# Convert both samples
sample_0 = convert_to_conversation_format(ds_batch[0])
sample_1 = convert_to_conversation_format(ds_batch[1])

# Create real batch with DIFFERENT samples
batch = [sample_0, sample_1]

# Output the batch
batch

[{'messages': [{'role': 'system',
    'content': [{'type': 'text',
      'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]},
   {'role': 'user',
    'content': [{'type': 'image', 'image': 'IMAGE_PLACEHOLDER'},
     {'type': 'text',
      'text': 'Detect the bounding box of the nutrition table.'}]},
   {'role': 'assistant',
    'content': [{'type': 'text',
      'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(14,57),(991,603)<|box_end|>'}]}],
  'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2592x1944>},
 {'messages': [{'role': 'system',
    'content': [{'type': 'text',
      'text': 'You are a Vision Language Model specialized in interpreting visua

### 📋 What to copy from above:
- It's a list of 2 dicts
- Each sample is DIFFERENT (different images, different bboxes)
- Each dict has 'messages' (with 'IMAGE_PLACEHOLDER') and 'image' (PIL Image)
- This is what gets passed to collator

## Stage 4: After restore_images_in_conversations()

This is the format that goes into `apply_chat_template` (PIL Images restored).

- **Input**: messages with 'IMAGE_PLACEHOLDER' (string)
- **Output**: messages with actual `<PIL.Image>` objects

This output format is what your instruction shows!

In [ ]:
# Extract messages and images
messages_list = [sample['messages'] for sample in batch]
images_list = [sample['image'] for sample in batch]

# Restore images in messages
restored = restore_images_in_conversations(messages_list, images_list)

# Output the restored format
restored

[[{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided product images and detect the nutrition tables in a certain format.\nFocus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary.'}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2592x1944>},
    {'type': 'text',
     'text': 'Detect the bounding box of the nutrition table.'}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '<|object_ref_start|>nutrition-table<|object_ref_end|><|box_start|>(14,57),(991,603)<|box_end|>'}]}],
 [{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You are a Vision Language Model specialized in interpreting visual data from product images.\nYour task is to analyze the provided

### 📋 What to copy from above:
- Copy the structure of `restored`
- Note: Now has actual PIL Images, not `'IMAGE_PLACEHOLDER'`
- This is what goes into `processor.apply_chat_template()`
- **CRITICAL**: Images must be PIL.Image objects at this stage!

## Summary - What You've Generated

You now have outputs for all stages:

1. **Raw sample** - from dataset
2. **Converted** - after `convert_to_conversation_format()`
3. **Batch** - list of samples
4. **Restored** - after `restore_images_in_conversations()`

### Next Steps:

1. Copy outputs from cells above
2. Paste into `tests/test_golden_output.py`:
   - Cell 2 output → `test_convert_to_conversation_format_golden_output()`
   - Cell 4 output → `test_restore_images_golden_output()`
3. Run the test to verify format stays consistent